In [83]:
!pip install crewai python-dotenv

In [84]:
!pip install -U litellm

  Using cached litellm-1.65.0-py3-none-any.whl.metadata (36 kB)
Using cached litellm-1.65.0-py3-none-any.whl (7.0 MB)
  Attempting uninstall: litellm
    Found existing installation: litellm 1.60.2
    Uninstalling litellm-1.60.2:
      Successfully uninstalled litellm-1.60.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.108.0 requires litellm==1.60.2, but you have litellm 1.65.0 which is incompatible.


In [85]:
!pip install crewai-tools

  Using cached litellm-1.60.2-py3-none-any.whl.metadata (36 kB)
Using cached litellm-1.60.2-py3-none-any.whl (6.7 MB)
  Attempting uninstall: litellm
    Found existing installation: litellm 1.65.0
    Uninstalling litellm-1.65.0:
      Successfully uninstalled litellm-1.65.0


In [86]:
!pip install langchain-groq

In [87]:

import os
import time
import litellm
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

In [88]:
from crewai import Agent, Task , Crew, LLM
from crewai_tools import SerperDevTool

In [89]:
from dotenv import load_dotenv
load_dotenv()

False

In [91]:

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY is missing! Set it in your .env file.")
if not SERPER_API_KEY:
    raise ValueError("❌ SERPER_API_KEY is missing! Add it to your .env file.")


In [92]:
topic = "Medical using Gen AI "

In [93]:
!pip install langchain-groq
from langchain_groq import ChatGroq


In [94]:
# ✅ Define LLM
# llm = ChatGroq(model="llama3-70b-8192", api_key=GROQ_API_KEY)
llm = LLM(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)


# ✅ Define Search Tool
search_tool = SerperDevTool(api_key=SERPER_API_KEY, n=10)

In [95]:
#Agent 1
senior_research_analyst=Agent(
    role="Senior Research Analyst",
    goal=f"Research, analyze and synthesize comprehensive information on {topic} from reliable web sources",
    backstory = "You are Senior Research Analyst with 5 years of experience in the field of AI and Machine Learning. You have worked on multiple projects related to the medical industry and have a good understanding of the domain.",
    allow_delegation=False,
    verbose=True,
    tools=[search_tool],
    llm=llm
)

In [96]:
#Agent 2
content_writer=Agent(
    role="Content Writer",
    goal=f"transform the research findings into engaging blog posts while maintaining accuracy",
    backstory = "You are a Content Writer with 3 years of experience in writing articles on various topics. You have written articles on the medical industry in the past and have a good understanding of the domain.",
    allow_delegation=False,
    verbose=True,
    tools=[search_tool],
    llm=llm
)

In [97]:
#Research Tasks
research_tasks= Task(
    description = (
        f"""
          1.Conduct comprehensive research on {topic} including:
            - Recent development and news
            - key inductry trends and innovations
            - Expert opinions ans analyses
            - Statistical data and market insights
          2. Evalute souce credibility and fact-check all information.
          3. Organize findings into a structured research brief.
          4. Include all relevant citations and sources.
        """
    ),
    expected_output=""" A detailed research report containing:
        - Executive summary of key finding,
        - Comprehensive analysis of current trends and developments
        - List of verified facts and statistics
        - All citations ans links to original sources
        - Clear cateogorization of main themes and patterns
        please format with clear sections and bullets points for easy reference.

    """,
    agent=senior_research_analyst
)

In [98]:
writing_tasks = Task(
    description=(
        """Using the research brief provided, create an engaging blog post that:
        1. Transforms technical information into accessible content.
        2. Maintains all factual accuracy and citations from the research.
        3. Includes:
          - Attention-grabbing introduction.
          - Well-structured body sections with clear headings.
          - Compelling conclusion.
        4. Preserves all source citations in [Source:URL] format.
        5. Includes a References section at the end."""
    ),
    expected_output=(
        """A polished blog post in markdown format that:
        - Engages readers while maintaining accuracy.
        - Contains properly structured sections.
        - Includes inline citations hyperlinked to the original source URL.
        - Presents information in an accessible yet informative way.
        - Follows proper markdown formatting, using H1 for the title and H3 for the subsections."""
    ),
    agent=content_writer
)

In [99]:
crew = Crew(
    agents=[senior_research_analyst, content_writer],
    tasks=[research_tasks, writing_tasks],
    verbose= True
)

In [100]:
import litellm
import os
import time

In [101]:
# ✅ Define a function to call Groq API
def call_groq():
    return litellm.completion(
        model="llama3-70b-8192",  # ✅ Ensure the model is correct
        api_key=GROQ_API_KEY,
        api_base="https://api.groq.com/openai/v1",
        messages=[  # ✅ Updated user message
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": "Tell me about Medical using Gen AI."}
        ]
    )

# ✅ Retry API Requests
def retry_request(func, retries=3, delay=5):
    for attempt in range(retries):
        try:
            return func()
        except litellm.APIError as e:
            print(f"⚠️ Attempt {attempt + 1} failed: {e}")
            time.sleep(delay)
    raise Exception("❌ All retry attempts failed.")

# ✅ Run CrewAI with Groq API
result = retry_request(call_groq)

print(result)


/usr/local/lib/python3.11/dist-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)


ModelResponse(id='chatcmpl-815f6698-85a4-4c19-a386-81b520477bf7', created=1743344014, model='groq/llama3-70b-8192', object='chat.completion', system_fingerprint='fp_2e0feca3c9', choices=[Choices(finish_reason='stop', index=0, message=Message(content="What a fascinating topic! Medical applications using Generative AI (Gen AI) are rapidly evolving and have the potential to revolutionize the healthcare industry. Here are some exciting developments and possibilities:\n\n**1. Medical Imaging Analysis:** Gen AI algorithms, such as ConvNets, can be trained to analyze medical images like X-rays, CT scans, and MRIs to detect abnormalities, tumors, and diseases like cancer, cardiovascular disease, and neurological disorders. This can help radiologists and doctors make more accurate diagnoses and reduce false positives.\n\n**2. Synthetic Medical Data Generation:** Gen AI can generate synthetic medical data, such as images, patient records, and genomic data, which can be used to train AI models, r

In [103]:
# Result = crew.kickoff(inputs={"topic": topic})

# print(Result)